In [3]:
#Librerias importadas 

import pandas as pd
import openpyxl as op
import numpy as np
import mysql.connector as conn
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
  

pd.set_option("display.max_rows",None)
pd.set_option("display.max_columns", None)

In [2]:
#Lectura de librerias

dftipos= pd.read_excel("transacciones de vivienda por tipo y barrio.xls")
dfmetros = pd.read_excel("transacciones vivienda por m2.xls")
dfprecio = pd.read_excel("precio medio metro.xls")      #
dftransmisiones = pd.read_excel("Transmisiones viviendas.xls")
dfhipotecas = pd.read_excel("Hipotecas madrid .xls")
dfpreciom2= pd.read_excel("pvp.xls")


In [3]:
#Dataframes finales


#dftipos
#dfmetros
#dfprecio
#dftransmisiones
#dfhipotecas
#dfpreciom2

In [4]:
#Cambia el nombre de las Columnas

dfmetros.rename({"Columna1":"Distrito","Columna2":"Total"},axis=1, inplace = True)
dfprecio.rename({"Columna1":"Distrito","Columna2":"Total","Columna3":"Nueva","Columna4":"Usada"},axis=1, inplace = True)

In [5]:
#Borro la primera row



dfprecio = dfprecio.iloc[1: , :]


In [6]:
#filtrado por indice

dfmetros = dfmetros.loc[dfmetros.Distrito != 'No consta']
dftipos = dftipos.loc[dftipos.Distrito != 'No consta']
dftransmisiones = dftransmisiones.loc[dftransmisiones.Distrito != 'No consta']

In [7]:
#Funcion de limpieza del Dataframe


def limpiar_distritos (my_dataframe, column_district_name):

    distrito = []  

    for e in my_dataframe[f"{column_district_name}"]:

        if e.startswith ('01'):
            distrito.append ('Centro')

        elif e.startswith ('02'):
            distrito.append ('Arganzuela')

        elif e.startswith ('03'):
            distrito.append ('Retiro')

        elif e.startswith ('04'):
            distrito.append ('Salamanca')

        elif e.startswith ('05'):
            distrito.append ('Chamartin')

        elif e.startswith ('06'):
            distrito.append ('Tetuán')

        elif e.startswith ('07'):
            distrito.append ('Chamberí')

        elif e.startswith ('08'):
            distrito.append ('Fuencarral-El Pardo')

        elif e.startswith ('09'):
            distrito.append ('Moncloa-Aravaca')

        elif e.startswith ('10'):
            distrito.append ('Latina')   

        elif e.startswith ('11'):
            distrito.append ('Carabanchel')

        elif e.startswith ('12'):
            distrito.append ('Usera')

        elif e.startswith ('13'):
            distrito.append ('Puente de Vallecas')

        elif e.startswith ('14'):
            distrito.append ('Moratalaz')

        elif e.startswith ('15'):
            distrito.append ('Ventas')

        elif e.startswith ('16'):
            distrito.append ('Hortaleza')

        elif e.startswith ('17'):
            distrito.append ('Villaverde')

        elif e.startswith ('18'):
            distrito.append ('Villaverde')

        elif e.startswith ('19'):
            distrito.append ('Villa de Vallecas')

        elif e.startswith ('20'):
            distrito.append ('San Blas - Canillejas')

        elif e.startswith ('21'):
            distrito.append ('Barajas')

        else:
            distrito.append ('Comunidad de Madrid')

    my_dataframe['Barrio']= distrito

    my_dataframe.Distrito= my_dataframe.Distrito.apply(lambda x: x.split('.')[1] if len(x.split('.'))==2 else x)
    return my_dataframe




In [8]:
#llamo a la función de limpieza


#dftipos = limpiar_distritos(dftipos,"Distrito")
#dfmetros = limpiar_distritos(dfmetros,"Distrito")
#dfprecio = limpiar_distritos(dfprecio,"Distrito")
#dftransmisiones = limpiar_distritos(dftransmisiones,"Distrito")
#dfhipotecas = limpiar_distritos(dfhipotecas,"Distrito")
dfpreciom2 = limpiar_distritos(dfpreciom2,"Distrito")

In [9]:
#merge todos los dataframe

df1 = pd.merge(dftipos, dfmetros, left_on = "Distrito", right_on = "Distrito")
df2 = pd.merge(df1, dfprecio, left_on = 'Distrito', right_on ='Distrito' )
df3 = pd.merge(df2, dftransmisiones, left_on = "Distrito", right_on = "Distrito")
dfinal = pd.merge(df3, dfhipotecas, left_on = "Distrito", right_on = "Distrito")


In [10]:
#Borrar las columnas duplicadas al mergear

#df1.drop(['Barrio_x', 'Barrio_y', 'Total'], axis=1, inplace=True)
#df2.drop(['Barrio_x', 'Barrio_y', 'Total_x'], axis=1, inplace=True)
#df3.drop(['Barrio_x', 'Barrio_y'], axis=1, inplace=True)
#df4.drop(['Barrio_x', 'Barrio_y', 'Barrio'], axis=1, inplace=True)
#dfinal.drop(['Barrio', 'A 20 años', ], axis=1, inplace=True)
#dfinal.drop(['Barrio', 'Total_y', 'Total_x'], axis=1, inplace=True)
#dfinal.drop(['Total_x', 'Barrio_y',"Total_y", "Barrio_x"], axis=1, inplace=True)


In [11]:
#Rename de las columnas


dfinal.rename(columns=({ 'Menos de 40': '40m2',
                        'A 20 años':'20_años',
                        '40 a 60': '40/60m2',
                        '60 a 80':'60/80m2',
                        '80_a_100':'80/100m2',
                        'Más_de_100':'100m2',
                        'Nueva':'vivienda_nueva_€/m2',
                        'Total hipoteca':'total_hipoteca',
                        'Tipo de interés':'tipo_de_interés',
                        'Usada':'vivienda_usada_€/m2' }), 
    inplace=True)

In [12]:
#Exportar el Dataframe final

dfinal.to_excel(r'/Users/usuario/Desktop/Ironhack/Real-Estate-Final-Project\Dataframefinal.xlsx', index = False)

In [13]:
#Renombro las columnas para SQL


dfinal = dfinal.rename(columns=({'Distrito':'distrito',
                         'Totales':'totales',
                         'Nuevas':'nuevas',
                         'Usadas':'usadas','Libres':'libres',
                         'vivienda nueva €/m2':'vivienda_nueva_euro/m2',
                         'vivienda usada €/m2)':'vivienda_usada_euro/m2',
                         'De 0 a 2':'0&2',
                         'De 2 a 5 ':'2&5',
                         'De 5 a 10':'5&10','Más de 10':'10_mas',
                         'total hipoteca':'total_hipoteca',
                         'tipo de interés':'tipo_de_interés',
                         '20 años':'20años'}), 
    )

In [14]:
#Función para crear BD en SQL


def create_db_sql(df):

    crea_db = conn.connect(host='localhost', user='root', passwd='Teresita123')
    cursor = crea_db.cursor()
    cursor.execute('drop database if exists real_estate') 
    cursor.execute('create database real_estate')  
    cursor.execute('show databases')
    for x in cursor:
        print(x)
    db = conn.connect(host='localhost', user='root', passwd='admin', database='real_estate')

    cursor.execute('drop table if exists real_estate')  

    cursor = db.cursor()
    tabla = '''
        create table real_estate(

            id int,
            distrito varchar(500),
            totales int,
            nuevas int,
            libres int(500),
            usadas int,
            40m2 int,
            40/60m2 int(500),
            60/80m2 int,
            80/100m2 int,
            80/100m2 int(500),
            100m2 int(500),
            vivienda_nueva_€/m2 float,
            vivienda_usada_€/m2 float,
            0&2 int,
            2&5 int,
            5&10 int,
            10_mas int ,
            total_hipoteca float,
            tipo_de_interés float,
            20años float,
            
            
            
            
            
            
        );
    '''

In [15]:
#Web scrapping Idealista con Selenium


In [16]:
url = 'https://www.idealista.com/sala-de-prensa/informes-precio-vivienda/venta/madrid-comunidad/report/'

In [17]:
dfpreciom2.head(2)    #Precio medio m2

,Distrito,2019,2020,Barrio
0,Ciudad de Madrid,3707,3546,Comunidad de Madrid
1,Centro,4995,4826,Centro


In [18]:
dftipos.head(2)   #Transmisiones de viviendas según tipología

,Distrito,Totales,Nuevas,Libres,VPO,Usadas
0,Ciudad de Madrid,25332,6070,5287,783,19262
1,01. Centro,1664,177,177,0,1487


In [19]:
dfmetros.head(2)   #ransmisiones de viviendas según m2

,Distrito,Total,Menos de 40,40 a 60,60 a 80,80 a 100,Más de 100
0,Ciudad de Madrid,25332,2040,5929,6254,4335,6774
1,01. Centro,1664,465,411,295,190,303


In [20]:
dfprecio.head(2)   #borrarlo

,Distrito,Total,Nueva,Usada
1,Ciudad de Madrid,3481.61,3176.5,3572.02
2,01. Centro,5223.73,5468.63,5194.16


In [21]:
dftransmisiones.head(2)   #Tran

,Distrito,Total,De 0 a 2,De 2 a 5,De 5 a 10,Más de 10
0,Ciudad de Madrid,13991,3304,1868,1705,7114
1,01. Centro,1081,239,170,173,499


In [22]:
dfhipotecas   #Hipoteca y tipo de interés

,Distrito,Total hipoteca,Tipo de interés,A 20 años
0,Ciudad de Madrid,196497.40,2.28,1020.18
1,01. Centro,219937.37,2.11,1123.87
2,011. Palacio,174941.09,2.19,900.91
3,012. Embajadores,154693.64,2.20,797.22
4,013. Cortes,263389.83,2.01,1333.61
5,014. Justicia,369451.06,1.98,1865.23
6,015. Universidad,245932.05,2.03,1247.90
7,016. Sol,350526.34,1.89,1754.97
8,02. Arganzuela,222079.51,1.99,1122.17
9,021. Imperial,201208.06,2.27,1043.77


,Distrito,Menos de 40,40 a 60,60 a 80,80 a 100,Más de 100
0,01. Centro,465,411,295,190,303
1,011. Palacio,30,38,34,21,22
2,012. Embajadores,247,169,127,46,57
3,013. Cortes,16,31,17,14,35
4,014. Justicia,45,43,32,41,57
5,015. Universidad,117,123,77,59,90
6,016. Sol,10,7,8,9,42
7,02. Arganzuela,78,161,245,186,261
8,021. Imperial,21,25,54,29,41
9,022. Las Acacias,9,26,13,10,9
